- <mark> 弹性网络模型效果差 </mark>
- <mark> 遍历所有可能？ </mark>
- <mark> 是否需要进行预剪枝，使用Boosting等操作 </mark>
- 集成算法和回归加权
## import包

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from xgboost import XGBRFClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm

## 数据准备

In [ ]:
# 读取数据
df_all = pd.read_csv('/Users/wangjingran/Desktop/PTEN mutation/L on ML/paras.txt', sep='\t')
print(df_all)

In [ ]:
# 读取数据
df_AA = pd.read_csv('itasser-AAN-feature.txt', sep='\t')
df_dyn = pd.read_csv('itasser_pos_dyn.txt', sep='\t')
##### 对氨基酸网络的特征进行ML
print("------------------------------氨基酸网络------------------------------")
# 数据集切分
# 疾病数据切分
df_AA_ASD_Cancer = df_AA[df_AA['Disease'].isin(['ASD', 'Cancer'])]
df_AA_ASD_AC = df_AA[df_AA['Disease'].isin(['ASD', 'ASD_Cancer'])]
df_AA_Cancer_AC = df_AA[df_AA['Disease'].isin(['Cancer', 'ASD_Cancer'])]

# 洗牌打乱    
X_1 = df_AA_ASD_Cancer[['Betweenness', 'Closeness', 'Degree', 'Eigenvector', 'Clustering_coefficient']]
y_1 = df_AA_ASD_Cancer['Disease']
# 切分后的数据集重新将index排序
X_1 = X_1.reset_index(drop=True)
y_1 = y_1.reset_index(drop=True)
shuffle_index = np.random.permutation(X_1.index)
X_1= X_1.iloc[shuffle_index]
y_1 = y_1.iloc[shuffle_index]

X_2 = df_AA_ASD_Cancer[['Betweenness', 'Closeness', 'Degree', 'Eigenvector', 'Clustering_coefficient']]
y_2 = df_AA_ASD_Cancer['Disease']
X_2 = X_2.reset_index(drop=True)
y_2 = y_2.reset_index(drop=True)
shuffle_index = np.random.permutation(X_2.index)
X_2 = X_2.iloc[shuffle_index]
y_2 = y_2.iloc[shuffle_index]

X_3 = df_AA_ASD_Cancer[['Betweenness', 'Closeness', 'Degree', 'Eigenvector', 'Clustering_coefficient']]
y_3 = df_AA_ASD_Cancer['Disease']
X_3 = X_3.reset_index(drop=True)
y_3 = y_3.reset_index(drop=True)
shuffle_index = np.random.permutation(X_3.index)
X_3 = X_3.iloc[shuffle_index]
y_3 = y_3.iloc[shuffle_index]

# 训练集测试集切分
# [806rows, 5cols]
X_train_1 = X_1[:650]
X_test_1 = X_1[650:]
y_train_1 = y_1[:650]
y_test_1 = y_1[650:]

X_train_2 = X_2[:650]
X_test_2 = X_2[650:]
y_train_2 = y_2[:650]
y_test_2 = y_2[650:]

X_train_3 = X_3[:650]
X_test_3 = X_3[650:]
y_train_3 = y_3[:650]
y_test_3 = y_3[650:]


# 模型构建
rf_clf_1 = AdaBoostClassifier(DecisionTreeClassifier(max_depth=1), n_estimators=10000, learning_rate=0.5, random_state=42)
rf_clf_2 = AdaBoostClassifier(DecisionTreeClassifier(max_depth=1), n_estimators=10000, learning_rate=0.5, random_state=42)
rf_clf_3 = AdaBoostClassifier(DecisionTreeClassifier(max_depth=1), n_estimators=10000, learning_rate=0.5, random_state=42)

rf_clf_1.fit(X_train_1, y_train_1)
rf_clf_2.fit(X_train_2, y_train_2)
rf_clf_3.fit(X_train_3, y_train_3)
score_1 = rf_clf_1.feature_importances_
score_2 = rf_clf_2.feature_importances_
score_3 = rf_clf_3.feature_importances_
print("对于ASD|Cancer重要性得分: ", end = '')
print(['Betweenness', 'Closeness', 'Degree', 'Eigenvector', 'Clustering_coefficient'])
print(score_1)
print("对于ASD|ASD_Cancer重要性得分: ", end = '')
print(['Betweenness', 'Closeness', 'Degree', 'Eigenvector', 'Clustering_coefficient'])
print(score_2)
print("对于Cancer|ASD_Cancer重要性得分: ", end = '')
print(['Betweenness', 'Closeness', 'Degree', 'Eigenvector', 'Clustering_coefficient'])
print(score_3)

# 模型评估
y_pred_1 = rf_clf_1.predict(X_test_1)
y_pred_2 = rf_clf_2.predict(X_test_2)
y_pred_3 = rf_clf_3.predict(X_test_3)
print('ASD|Cancer预测准确度是: ', end='')
print(accuracy_score(y_test_1, y_pred_1))
print('ASD|ASD_Cancer预测准确度是: ', end='')
print(accuracy_score(y_test_2, y_pred_2))
print('Cancer|Control预测准确度是: ', end='')
print(accuracy_score(y_test_3, y_pred_3))

## 绘制ROC曲线

## 定义函数

In [ ]:

def ROC(fpr, tpr, roc_auc, color, label):
    plt.figure()
    plt.plot(fpr, tpr, color=color, lw=2, label= label + '\n' + 'AUC = %0.2f' % roc_auc)
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('FPR')
    plt.ylabel('TPR')
    plt.title(label)
    plt.legend(loc="lower right")
    plt.show()


## 准备数据
- 注意数据需要转化成binary

In [ ]:
label_encoder = LabelEncoder()
y_test_1_binary = label_encoder.fit_transform(y_test_1)
y_pred_1_binary = label_encoder.fit_transform(y_pred_1)
y_test_2_binary = label_encoder.fit_transform(y_test_2)
y_pred_2_binary = label_encoder.fit_transform(y_pred_2)
y_test_3_binary = label_encoder.fit_transform(y_test_3)
y_pred_3_binary = label_encoder.fit_transform(y_pred_3)
y_test_4_binary = label_encoder.fit_transform(y_test_4)
y_pred_4_binary = label_encoder.fit_transform(y_pred_4)
y_test_5_binary = label_encoder.fit_transform(y_test_5)
y_pred_5_binary = label_encoder.fit_transform(y_pred_5)
y_test_6_binary = label_encoder.fit_transform(y_test_6)
y_pred_6_binary = label_encoder.fit_transform(y_pred_6)

fpr_1, tpr_1, thresholds_1 = roc_curve(y_test_1_binary, y_pred_1_binary)
roc_auc_1 = auc(fpr_1, tpr_1)
fpr_2, tpr_2, thresholds_2 = roc_curve(y_test_2_binary, y_pred_2_binary)
roc_auc_2 = auc(fpr_2, tpr_2)
fpr_3, tpr_3, thresholds_3 = roc_curve(y_test_3_binary, y_pred_3_binary)
roc_auc_3 = auc(fpr_3, tpr_3)
fpr_4, tpr_4, thresholds_4 = roc_curve(y_test_4_binary, y_pred_4_binary)
roc_auc_4 = auc(fpr_4, tpr_4)
fpr_5, tpr_5, thresholds_5 = roc_curve(y_test_5_binary, y_pred_5_binary)
roc_auc_5 = auc(fpr_5, tpr_5)
fpr_6, tpr_6, thresholds_6 = roc_curve(y_test_6_binary, y_pred_6_binary)
roc_auc_6 = auc(fpr_6, tpr_6)

## 开始画图

In [ ]:
ROC(fpr_1, tpr_1, roc_auc_1, 'b', 'AA-ASD|Cancer')
ROC(fpr_2, tpr_2, roc_auc_2, 'r', 'AA-ASD|ASD_Cancer')
ROC(fpr_3, tpr_3, roc_auc_3, 'g', 'AA-Cancer|ASD_Cancer')
ROC(fpr_4, tpr_4, roc_auc_4, 'orange', 'dyn-ASD|Cancer')
ROC(fpr_5, tpr_5, roc_auc_5, 'purple', 'dyn-ASD|ASD_Cancer')
ROC(fpr_6, tpr_6, roc_auc_6, 'pink', 'dyn-Cancer|ASD_Cancer')

## 模型循环选择——ASD versus Cancer

In [ ]:
# 生成全部的组合
import itertools
variables = ['Betweenness', 'Closeness', 'Degree', 'Eigenvector', 'Clustering_coefficient']
all_combinations = []
# 组合
for r in range(1, len(variables)+1):
    combinations = list(itertools.combinations(variables, r))
    combinations = [list(combination) for combination in combinations]  # 转换为列表
    all_combinations.extend(combinations)

for combination in all_combinations:
    print(combination)
    # 洗牌打乱    
    X_select = df_AA_ASD_Cancer[combination]
    y_select = df_AA_ASD_Cancer['Disease']
    # 切分后的数据集重新将index排序
    X_select = X_select.reset_index(drop=True)
    y_select = y_select.reset_index(drop=True)
    shuffle_index = np.random.permutation(X_select.index)
    X_select= X_select.iloc[shuffle_index]
    y_select = y_select.iloc[shuffle_index]
    # 切分数据集
    X_train_select = X_select[:650]
    X_test_select = X_select[650:]
    y_train_select = y_select[:650]
    y_test_select = y_select[650:]
    # 训练模型
    rf_clf_select = AdaBoostClassifier(DecisionTreeClassifier(max_depth=1), n_estimators=2000, learning_rate=0.5, random_state=42)
    rf_clf_select.fit(X_train_select, y_train_select)
    score_select = rf_clf_select.feature_importances_
    y_pred_select = rf_clf_select.predict(X_test_select)
    # 打印输出
    print("-"*20,end = '')
    print(str(combination),end = '')
    print("-"*20)
    print("*****重要性得分*****")
    print(score_select)
    print("*****模型准确性*****")
    print(accuracy_score(y_test_select, y_pred_select))



## 模型循环选择——ASD versus ASD_Cancer

In [ ]:
# 生成全部的组合
import itertools
variables = ['Co.evolution','Conservation','Entropy','RASA','energy',
             'Betweenness','Closeness','Degree','Eigenvector',
             'Clustering.coefficient','Effectiveness','Sensitivity',
             'MSF','DFI','Stiffness']
all_combinations = []
# 组合
for r in range(1, len(variables)+1):
    combinations = list(itertools.combinations(variables, r))
    combinations = [list(combination) for combination in combinations]  # 转换为列表
    all_combinations.extend(combinations)

list_ASD_AC = []
df_AA_ASD_Cancer = df_all[df_all['Disease'].isin(['ASD', 'Cancer'])]
df_AA_ASD_AC = df_all[df_all['Disease'].isin(['ASD', 'ASD_Cancer'])]
df_AA_Cancer_AC = df_all[df_all['Disease'].isin(['Cancer', 'ASD_Cancer'])]

for combination in tqdm(all_combinations):
    # 洗牌打乱    
    X_select = df_AA_ASD_AC[combination]
    y_select = df_AA_ASD_AC['Disease']
    # 切分后的数据集重新将index排序
    X_select = X_select.reset_index(drop=True)
    y_select = y_select.reset_index(drop=True)
    shuffle_index = np.random.permutation(X_select.index)
    X_select= X_select.iloc[shuffle_index]
    y_select = y_select.iloc[shuffle_index]
    # 切分数据集
    X_train_select = X_select[:round(X_select.shape[0] * 0.8)]
    X_test_select = X_select[round(X_select.shape[0] * 0.8):]
    y_train_select = y_select[:round(X_select.shape[0] * 0.8)]
    y_test_select = y_select[round(X_select.shape[0] * 0.8):]
    # 训练模型
    rf_clf_select = XGBRFClassifier(DecisionTreeClassifier(max_depth=1),
                                        n_estimators=2000, learning_rate=0.5, 
                                        random_state=42)
    rf_clf_select.fit(X_train_select, y_train_select)
    score_select = rf_clf_select.feature_importances_
    y_pred_select = rf_clf_select.predict(X_test_select)
    
    # 计算ROC值
    label_encoder = LabelEncoder()
    y_test_select_bi = label_encoder.fit_transform(y_test_select)
    y_pred_select_bi = label_encoder.fit_transform(y_pred_select)
    fpr_select, tpr_select, thresholds_select = roc_curve(y_test_select_bi, y_pred_select_bi)
    roc_auc_select = auc(fpr_select, tpr_select)

    # 打印输出
    print("-"*20,end = '')
    print(str(combination),end = '')
    print("-"*20)
    print("*****重要性得分*****")
    print(score_select)
    print("*****模型准确性*****")
    print(accuracy_score(y_test_select, y_pred_select))
    print("*****ROC值*****")
    print(roc_auc_select)
    # 结果输入
    list_ASD_AC.append([str(combination), score_select, roc_auc_select])
    

- 写入文件

In [ ]:
f = open("ASD versus ASD_Cancer.txt", "w")
for i in list_ASD_AC:
    for j in i:
        f.write(str(j)+ '\t')
    f.write("\n")
f.close()

## 模型循环选择——Cancer versus ASD_Cancer

In [ ]:
# 生成全部的组合
import itertools
variables = ['Co.evolution','Conservation','Entropy','RASA','energy',
             'Betweenness','Closeness','Degree','Eigenvector',
             'Clustering.coefficient','Effectiveness','Sensitivity',
             'MSF','DFI','Stiffness']
all_combinations = []
# 组合
for r in range(1, len(variables)+1):
    combinations = list(itertools.combinations(variables, r))
    combinations = [list(combination) for combination in combinations]  # 转换为列表
    all_combinations.extend(combinations)

list_Cancer_AC = []
df_AA_ASD_Cancer = df_all[df_all['Disease'].isin(['ASD', 'Cancer'])]
df_AA_ASD_AC = df_all[df_all['Disease'].isin(['ASD', 'ASD_Cancer'])]
df_AA_Cancer_AC = df_all[df_all['Disease'].isin(['Cancer', 'ASD_Cancer'])]

for combination in tqdm(all_combinations[20000:]):
    # 洗牌打乱    
    X_select = df_AA_Cancer_AC[combination]
    y_select = df_AA_Cancer_AC['Disease']
    # 切分后的数据集重新将index排序
    X_select = X_select.reset_index(drop=True)
    y_select = y_select.reset_index(drop=True)
    shuffle_index = np.random.permutation(X_select.index)
    X_select= X_select.iloc[shuffle_index]
    y_select = y_select.iloc[shuffle_index]
    # 切分数据集
    X_train_select = X_select[:round(X_select.shape[0] * 0.8)]
    X_test_select = X_select[round(X_select.shape[0] * 0.8):]
    y_train_select = y_select[:round(X_select.shape[0] * 0.8)]
    y_test_select = y_select[round(X_select.shape[0] * 0.8):]
    #print(y_test_select)
    # 训练模型
    y_train_select_bi = label_encoder.fit_transform(y_train_select)
    rf_clf_select = XGBRFClassifier(n_estimators=1000, n_jobs=-1, max_depth=5,oob_score=True)
    rf_clf_select.fit(X_train_select, y_train_select_bi)
    score_select = rf_clf_select.feature_importances_
    y_pred_select = rf_clf_select.predict(X_test_select)
    
    # 计算ROC值
    label_encoder = LabelEncoder()
    y_test_select_bi = label_encoder.fit_transform(y_test_select)
    y_pred_select_bi = label_encoder.fit_transform(y_pred_select)
    fpr_select, tpr_select, thresholds_select = roc_curve(y_test_select_bi, y_pred_select_bi)
    roc_auc_select = auc(fpr_select, tpr_select)

    # 打印输出
    #print("-"*20,end = '')
    #print(str(combination),end = '')
    #print("-"*20)
    #print("*****重要性得分*****")
    #print(score_select)
    #print("*****模型准确性*****")
    #print(accuracy_score(y_test_select, y_pred_select))
    #print("*****ROC值*****")
    #print(roc_auc_select)
    # 结果输入
    list_Cancer_AC.append([str(combination), score_select, roc_auc_select])
   

In [18]:
f = open("Cancer versus ASD_Cancer.txt", "w")
for i in list_Cancer_AC:
    for j in i:
        f.write(str(j)+ '\t')
    f.write("\n")
f.close()